This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key. Below is a way to store that file without uploading it publicly.

In [1]:
# Import the Tkinter and JSON libraries
from tkinter import *
from tkinter import filedialog
from tkinter import Tk
import json

# Quickly get rid of the root window popup
root = Tk()
root.withdraw()

# Use Filedialog.askopenfilename to open a dialog window 
# where you can select your credentials file.
filepath = filedialog.askopenfilename()
file = open(filepath, 'r')

# Open the credentials file with json.load
credentials = json.load(file)
file.close()

# Assign api_key to variable
api_key = credentials['API_Key']

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2014-01-01&end_date=2014-01-01&api_key={api_key}"
r = requests.get(url)
json_data = r.json()
json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2014-01-01',
  'end_date': '2014-01-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [4]:
# 1. Collect data for year 2017

url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31.json?api_key={api_key}"
r = requests.get(url)
json_data = r.json()

In [5]:
# 2. Convert json object to python dict
afx_dict = dict(json_data)

In [6]:
# 3. Calculate highest open price
# "if != None required to circumnavigate null values"
max_open_price = max([x[1] for x in json_data["dataset"]["data"] if x[1] != None])

In [7]:
print(max_open_price)

53.11


In [8]:
# 3. Calculate lowest price open price
min_open_price = min([x[1] for x in json_data["dataset"]["data"] if x[1] != None])

In [9]:
print(min_open_price)

34.0


In [10]:
# 4. Calculate the highest change in price in any one day
daily_change = []
for x in afx_dict["dataset"]["data"]:
    if x[2] != None and x[3] != None:
        daily_change.append(x[2] - x[3])
highest_daily_change = max(daily_change)

In [11]:
print(highest_daily_change)

2.8100000000000023


In [12]:
# 5. Calculate the price change in closing days
close_change = []

for i in range(len(afx_dict["dataset"]["data"])-2): # -2 added to avoid index error
    close_change.append(afx_dict["dataset"]["data"][i+2][4] - afx_dict["dataset"]["data"][i][4])

In [13]:
highest_close_change = max(close_change)
print(highest_close_change)

3.1499999999999986


In [14]:
# 6. Mean trading volume
trade_volume = []

for x in afx_dict["dataset"]["data"]:
    trade_volume.append(x[6])

In [15]:
from statistics import mean

In [16]:
print(mean(trade_volume))

89124.33725490196


In [17]:
#7. What was the median trading volume during this year
from statistics import median

In [18]:
print(median(trade_volume))

76286.0
